# Tavily Search as a Tool in Nebius Token Factory

[![](https://img.shields.io/badge/Powered%20by-Nebius%20AI-orange?style=flat&labelColor=darkblue&color=green)](http://tokenfactory.nebius.com/)

This notebook demonstrates how to use [Tavily Search](https://tavily.com/) as a tool with function calling on the Nebius Token Factory API. The LLM decides when to search the web and processes the results to answer user questions with up-to-date information.

## Prerequisites

- **Nebius API key** - Sign up for free at [Token Factory](https://tokenfactory.nebius.com/)
- **Tavily API key** - Get yours at [app.tavily.com](https://app.tavily.com) (1,000 free API credits/month)

## 1 - Setup

### 1.1 - If running on Google Colab

Add `NEBIUS_API_KEY` and `TAVILY_API_KEY` to **Secrets**

![](https://github.com/nebius/token-factory-cookbook/raw/main/images/google-colab-1.png)


### 1.2 - If running locally

Create an `.env` file with both keys:

```text
NEBIUS_API_KEY=your_nebius_api_key_here
TAVILY_API_KEY=tvly-your_tavily_api_key_here
```

## 2 - Install Dependencies

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    print("Running on Colab")
    RUNNING_ON_COLAB = True
else:
    print("NOT running on Colab")
    RUNNING_ON_COLAB = False

NOT running on Colab


In [2]:
%pip install -q openai python-dotenv pydantic tavily-python

You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## 3 - Load Configuration

In [3]:
import os

if RUNNING_ON_COLAB:
    from google.colab import userdata
    NEBIUS_API_KEY = userdata.get('NEBIUS_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv()
    NEBIUS_API_KEY = os.getenv('NEBIUS_API_KEY')
    TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

if NEBIUS_API_KEY:
    print('\u2705 NEBIUS_API_KEY found')
    os.environ['NEBIUS_API_KEY'] = NEBIUS_API_KEY
else:
    raise RuntimeError('\u274c NEBIUS_API_KEY NOT found')

if TAVILY_API_KEY:
    print('\u2705 TAVILY_API_KEY found')
    os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
else:
    raise RuntimeError('\u274c TAVILY_API_KEY NOT found')

✅ NEBIUS_API_KEY found
✅ TAVILY_API_KEY found


## 4 - Pick a Model

We need a model that supports function calling.

1. Go to **models** tab in [tokenfactory.nebius.com](https://tokenfactory.nebius.com/)
2. Select **text to text** models
3. Select a model **function calling** capability
4. Copy the model name

In [4]:
MODEL = "moonshotai/Kimi-K2.5"

## 5 - Define the Tavily Search Tool

We define `tavily_web_search` as a callable tool, then describe its schema using [Pydantic](https://docs.pydantic.dev/) so the LLM knows how to invoke it.

The function wraps `TavilyClient.search()` and returns the raw JSON response for the LLM to process.

In [16]:
import json
from pydantic import BaseModel, Field
from typing import Optional, Literal
from tavily import TavilyClient

# Initialize the Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)


# --- Pydantic schema for the tool parameters ---
class TavilyWebSearchParams(BaseModel):
    query: str = Field(
        ...,
        description="The search query. Keep under 400 characters. Use a focused search query, not a long prompt."
    )
    search_depth: Literal["basic", "advanced"] = Field(
        default="advanced",
        description="Search depth: 'basic' for general results, 'advanced' for highest relevance."
    )
    max_results: int = Field(
        default=5,
        description="Maximum number of search results to return (1-20)."
    )


# --- The actual tool function ---
def tavily_web_search(query: str, search_depth: str = "advanced",
                      topic: str = "general", max_results: int = 5,
                      time_range: str = None) -> str:
    """Search the web using Tavily and return raw results."""
    response = tavily_client.search(
        query=query,
        search_depth=search_depth,
        topic=topic,
        max_results=max_results,
        time_range=time_range,
    )

    return json.dumps(response)


# --- OpenAI-compatible tool definition ---
tools = [{
    "type": "function",
    "function": {
        "name": "tavily_web_search",
        "description": (
            "Search the web for current information using Tavily. "
            "Use this to answer questions that require up-to-date information, "
            "recent events, facts you are uncertain about, or anything beyond your training data."
        ),
        "parameters": TavilyWebSearchParams.model_json_schema()
    }
}]

available_tools = {"tavily_web_search": tavily_web_search}

print("Tavily search tool defined")
print(f"\nTool schema:\n{json.dumps(tools, indent=2)}")

Tavily search tool defined

Tool schema:
[
  {
    "type": "function",
    "function": {
      "name": "tavily_web_search",
      "description": "Search the web for current information using Tavily. Use this to answer questions that require up-to-date information, recent events, facts you are uncertain about, or anything beyond your training data.",
      "parameters": {
        "properties": {
          "query": {
            "description": "The search query. Keep under 400 characters. Use a focused search query, not a long prompt.",
            "title": "Query",
            "type": "string"
          },
          "search_depth": {
            "default": "advanced",
            "description": "Search depth: 'basic' for general results, 'advanced' for highest relevance.",
            "enum": [
              "basic",
              "advanced"
            ],
            "title": "Search Depth",
            "type": "string"
          },
          "max_results": {
            "default": 5,


## 6 - Initialize the LLM Client

In [17]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.tokenfactory.nebius.com/v1/",
    api_key=NEBIUS_API_KEY,
)

print(f"OpenAI client ready (model: {MODEL})")

OpenAI client ready (model: moonshotai/Kimi-K2.5)


## 7 - Tool Calling Loop

This is the core pattern: we send a user question to the LLM with the tool definitions, then handle any tool calls the model makes. The loop continues until the model produces a final text response.

```
User Question
     ↓
LLM (with tools) ─────→ Tool Call? ── Yes ──→ Execute tavily_web_search()
     ↑                                              │
     └────────────── Feed results back ──────────┘
     │
     No ──→ Final Answer
```

In [18]:
def ask(user_question: str, system_prompt: str = None, verbose: bool = True) -> str:
    """
    Send a question to the LLM with Tavily search available as a tool.
    Handles the full tool-calling loop.
    """
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_question})

    if verbose:
        print(f"\U0001f4ac User: {user_question}\n")

    while True:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
        )

        choice = response.choices[0]

        # If the model wants to call tools
        if choice.message.tool_calls:
            # Append the assistant message with tool calls
            messages.append({
                "role": "assistant",
                "tool_calls": choice.message.tool_calls
            })

            for call in choice.message.tool_calls:
                tool_fn = available_tools[call.function.name]
                args = json.loads(call.function.arguments)

                if verbose:
                    print(f"\U0001f50d Tool call: {call.function.name}")
                    print(f"   Args: {json.dumps(args, indent=2)}")

                # Execute the tool
                result = tool_fn(**args)

                if verbose:
                    # Show a preview of the results
                    preview = result[:300] + "..." if len(result) > 300 else result
                    print(f"   Result preview: {preview}\n")

                # Feed the tool result back to the LLM
                messages.append({
                    "role": "tool",
                    "content": result,
                    "tool_call_id": call.id,
                    "name": call.function.name
                })
        else:
            # No tool calls - we have the final answer
            answer = choice.message.content
            if verbose:
                print(f"\U0001f916 Assistant:\n{answer}")
            return answer

## 8 - Try It Out

### Example 1: Current events

Ask a question that requires recent information the LLM wouldn't have in its training data.

In [19]:
%%time

answer = ask("What are the latest developments in AI in February 2026?")

💬 User: What are the latest developments in AI in February 2026?

🔍 Tool call: tavily_web_search
   Args: {
  "query": "AI artificial intelligence latest developments February 2026",
  "max_results": 10,
  "search_depth": "advanced"
}
   Result preview: {"query": "AI artificial intelligence latest developments February 2026", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.marketingprofs.com/opinions/2026/54304/ai-update-february-13-2026-ai-news-and-views-from-the-past-week", "title": "AI Update, February...

🤖 Assistant:
 Based on the latest search results, here are the major AI developments taking place in February 2026:

## 🚀 Major Model Releases (Early February)

**February 5-7 saw significant model launches:**
- **Anthropic Claude Opus 4.6** (Feb 5): Features a 1 million-token context window, enhanced task planning, and multi-agent collaboration for parallel workflows
- **OpenAI GPT-5.3-Codex** (Feb 7): Focused on coding capabilities
- *

### Example 2: Factual lookup

A question where the model should search to verify up-to-date facts.

In [20]:
%%time

answer = ask("Who won the 2026 Super Bowl and what was the final score and who prefromed at halftime?")

💬 User: Who won the 2026 Super Bowl and what was the final score and who prefromed at halftime?

🔍 Tool call: tavily_web_search
   Args: {
  "query": "2026 Super Bowl winner final score halftime performer",
  "max_results": 5,
  "search_depth": "advanced"
}
   Result preview: {"query": "2026 Super Bowl winner final score halftime performer", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.foxsports.com/stories/nfl/2026-super-bowl-halftime-show-who-performing-time-more", "title": "2026 Super Bowl Halftime Show: Bad Bunny Perform...

🤖 Assistant:
 Based on the search results, here are the details for the **2026 Super Bowl (Super Bowl LX)**:

**Winner:** The **Seattle Seahawks** defeated the New England Patriots

**Final Score:** **Seattle Seahawks 29, New England Patriots 13**

**Halftime Performer:** **Bad Bunny** headlined the halftime show, making history as one of the youngest performers to do so. He was joined on stage by **Lady Gaga** and

### Example 3: Technical research

A developer question that benefits from current web search.

In [22]:
%%time

answer = ask(
    "What are the key differences between Llama 4 and Qwen3?",
    system_prompt="You are a helpful AI assistant with access to web search. Cite your sources with URLs."
)

💬 User: What are the key differences between Llama 4 and Qwen3?

🔍 Tool call: tavily_web_search
   Args: {
  "query": "Llama 4 vs Qwen3 comparison key differences 2024 2025",
  "max_results": 7,
  "search_depth": "advanced"
}
   Result preview: {"query": "Llama 4 vs Qwen3 comparison key differences 2024 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://magazine.sebastianraschka.com/p/the-big-llm-architecture-comparison", "title": "The Big LLM Architecture Comparison - Ahead of AI", "content": "L...

🔍 Tool call: tavily_web_search
   Args: {
  "query": "Meta Llama 4 features architecture availability 2024 2025",
  "max_results": 7,
  "search_depth": "advanced"
}
   Result preview: {"query": "Meta Llama 4 features architecture availability 2024 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://gpt-trainer.com/blog/llama+4+evolution+features+comparison", "title": "Llama 4: Meta's New AI Model - Evolu

### Example 4: No search needed

For questions the model can answer from training data, it should skip the tool call entirely.

In [23]:
%%time

answer = ask("What is the capital of France?")

💬 User: What is the capital of France?

🤖 Assistant:
 The capital of France is **Paris**.
CPU times: user 9.74 ms, sys: 8.76 ms, total: 18.5 ms
Wall time: 2.74 s


## 9 - Try Your Own Queries

In [ ]:
answer = ask("YOUR QUESTION HERE")